In [6]:
import numpy as np
import pandas as pd
#from __future__ import division

#set seed for reproducible use of PRNG
np.random.seed(seed=42)

#set length of dummy data set
length = 5

#define dummy input data and dummy variables for demand

ID = np.arange(length)
population = np.random.randint(low=4000, high=10000, size=length)
area = np.random.randint(low=1, high=50, size=length)
user_demand = [4]*length
penetration = 0.8
OBF = 50

#define demand function
def calc_demand (population, area, user_demand, penetration):
    users = population*penetration
    density = np.round(users / area, 2) #km2
    user_throughput = user_demand #temporary calculation!!
    capacity_required_km2 = user_throughput * density
    area_demand = np.round(capacity_required_km2 / OBF, 2)
    return pd.DataFrame({'ID':ID,
                         'population':population, 
                         'area': area,
                         'users': users, 
                         'density': density,
                         'user_throughput': user_throughput, 
                         'area_demand': area_demand})

#define dummy input data and dummy variables for supply
spectrum_bandwidth = [20]*length
sites = np.random.randint(low=1, high=40, size=length)
efficiency = [3]*length

def calc_supply (spectrum_bandwidth, sites, efficiency):
    cells = sites * 3
    total_capacity = spectrum_bandwidth * sites * cells * efficiency/1000
    area_supply = np.round(total_capacity / area, 0)
    return pd.DataFrame({'sites':sites, 
                         'cells': cells,
                         'area_supply': area_supply,
                        'new_sites': '' })

def calc_margin (demand, supply):
    margin = supply - demand
    return margin

#when you call the function, store the output as an object 
area_demand = calc_demand(population, area, user_demand, penetration)
area_supply = calc_supply(spectrum_bandwidth, sites, efficiency)
margin = calc_margin(area_demand.area_demand, area_supply.area_supply)

#margin.head(n=5)   

#concat all data
all_data = pd.concat([area_demand, area_supply, margin], axis=1)

#put a name on the margin column as it was previously not named
all_data.rename(columns={0:'margin'}, inplace=True)

lookup_df = pd.DataFrame({'spectrum_lookup' : (2,4,6,8,10,12,14,16),
                        'sites_lookup' : (5,10,15,20,25,30,35,40),
                        'supply_lookup' : (50,100,150,200,250,300,350,400)})

def lookup_new_sites(row):
    if row['margin'] < 0:
        # get the minimum number of sites that cover the current supply deficit
        number_new = lookup_df.loc[lookup_df['supply_lookup'] >= abs(row['area_demand']), 'sites_lookup'].values[0]
        return number_new - row['sites']
    else:
        return 0
    
all_data['new_sites'] = all_data.apply(lookup_new_sites, axis=1)

#print(all_data)  


# data = pd.DataFrame({'ID' : (1, 2),
#                     'area': (2, 3),
#                     'population' : (100, 200),
#                     'demand' : (100, 200)})
# print(data)  
# output = pd.DataFrame({'ID': (1,2,1,2,1,2),
#                     'year': (1,1,2,2,3,3),
#                     'area': (2,3,2,3,2,3),
#                     'population': (100,200,110,220,121,242),
#                     'demand': (100,200,120,240,144,288)})
# print(output)  



    

   ID  area  demand  population
0   1     2     100         100
1   2     3     200         200
   ID  area  demand  population  year
0   1     2     100         100     1
1   2     3     200         200     1
2   1     2     120         110     2
3   2     3     240         220     2
4   1     2     144         121     3
5   2     3     288         242     3
